# Lab 2

In [1]:
import numpy as np

In [60]:
def split_to_halves(p):
    sum_prob_half = sum(p) / 2
    i, left_sum = 1, p[0]
    while left_sum < sum_prob_half:
        left_sum += p[i]
        i += 1
    return i

def shannon_fano_encode(p, l=0, r=None):
    if r is None: 
        r = len(p)
    if r - l == 1:
        return p[l]
    elif r - l == 2:
        return {0: p[l], 1: p[r-1]}
    
    i = l + split_to_halves(p[l:r])
    return {0: shannon_fano_encode(p, l, i),
            1: shannon_fano_encode(p, i, r)}


In [61]:
prob = [1/16, 3/16, 3/16, 9/16]
prob = sorted(prob, reverse=True)
shannon_fano_encode(prob)

{0: 0.5625, 1: {0: {0: 0.1875, 1: 0.1875}, 1: 0.0625}}

In [62]:
p1 = [1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]
shannon_fano_encode(p1)

{0: {0: {0: 0.125, 1: 0.125}, 1: {0: 0.125, 1: 0.125}},
 1: {0: {0: 0.125, 1: 0.125}, 1: {0: 0.125, 1: 0.125}}}

In [39]:
def huffman(probs):
    """
    :param probs: list of probabilities
    """
    assert np.allclose(np.sum(probs), 1), 'Probabilities does not sum up to 1'
    for i in range(len(probs)):
        probs[i] = (probs[i], chr(97 + i), dict())
    
    while len(probs) > 1:
        probs.sort()
        
        (a_prob, a, a_tree), (b_prob, b, b_tree) = probs[:2]
        ab_prob = a_prob + b_prob
        ab = a + b
        ab_tree = dict()
        
        if len(a_tree) == 0:
            ab_tree[0] = a
        else:
            ab_tree[0] = a_tree
        
        if len(b_tree) == 0:
            ab_tree[1] = b
        else:
            ab_tree[1] = b_tree
        
        probs.pop(0)
        probs.pop(0)
        
        probs.append((ab_prob, ab, ab_tree))
        
    return probs[0][-1]

In [56]:
def retrieve_words(prefix_tree):
    code_word = ''
    word_dict = dict()
    def _retrieve_words(prefix_tree, code_word, word_dict):
        try:
            left, right = prefix_tree[0], prefix_tree[1]
            if type(left) == str:
                word_dict[left] = code_word + "0"
            else:
                _retrieve_words(left, code_word + "0", word_dict)

            if type(right) == str:
                word_dict[right] = code_word + "1"
            else:
                _retrieve_words(right, code_word + "1", word_dict)

            return word_dict
        except Exception as e:
            print('Exception: {}'.format(e))
    
    return _retrieve_words(prefix_tree, code_word, word_dict)

In [57]:
prefix_tree = huffman([0.1, 0.2, 0.3, 0.4])
prefix_tree

{0: 'd', 1: {0: 'c', 1: {0: 'a', 1: 'b'}}}

In [58]:
retrieve_words(prefix_tree)

{'d': '0', 'c': '10', 'a': '110', 'b': '111'}

# Test

## Test 1: lab 1, variant 26 (and 20)

In [59]:
probs = [0.03, 0.12, 0.01, 0.01, 0.03, 0.29, 0.05, 0.09, 0.08, 0.07, 0.07, 0.05, 0.04, 0.03, 0.02, 0.01]
prefix_tree = huffman(probs)
print(prefix_tree)

{0: {0: {0: 'h', 1: {0: 'g', 1: 'l'}}, 1: {0: {0: {0: 'o', 1: 'a'}, 1: {0: 'e', 1: 'n'}}, 1: 'b'}}, 1: {0: 'f', 1: {0: {0: 'j', 1: 'k'}, 1: {0: {0: {0: 'p', 1: {0: 'c', 1: 'd'}}, 1: 'm'}, 1: 'i'}}}}


In [61]:
retrieve_words(prefix_tree)

{'h': '000',
 'g': '0010',
 'l': '0011',
 'o': '01000',
 'a': '01001',
 'e': '01010',
 'n': '01011',
 'b': '011',
 'f': '10',
 'j': '1100',
 'k': '1101',
 'p': '111000',
 'c': '1110010',
 'd': '1110011',
 'm': '11101',
 'i': '1111'}

## Test 2: lecture 4

In [62]:
probs = [0.05, 0.15, 0.05, 0.4, 0.2, 0.15]
prefix_tree = huffman(probs)
print(prefix_tree)

{0: 'd', 1: {0: {0: {0: 'a', 1: 'c'}, 1: 'b'}, 1: {0: 'f', 1: 'e'}}}


In [63]:
retrieve_words(prefix_tree)

{'d': '0', 'a': '1000', 'c': '1001', 'b': '101', 'f': '110', 'e': '111'}